In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

#load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
BW=pd.read_csv('../data/BW_EN_tweets.csv')
CR=pd.read_csv('..data/CR_en_id.csv')

In [ ]:
BW_sents=list(BW.full_text)
CR_sents=list(CR.text)

In [ ]:
#Compute embedding for both lists
BW_embeddings = model.encode(BW_sents, convert_to_tensor=True)
CR_embeddings = model.encode(CR_sents, convert_to_tensor=True)

In [ ]:
#Compute cosine-similarits

cosine_scores = util.cos_sim(BW_embeddings, CR_embeddings)


In [ ]:
cosine_scores.shape

torch.Size([8830, 38482])

In [ ]:
sim_map = {}
arg=set()
count=0
for i in range(len(cosine_scores)):
    max_score = round(np.max(cosine_scores[i].detach().cpu().numpy()),1)
    max_arg = np.argmax(cosine_scores[i].detach().cpu().numpy())
    if max_score>=0.8: 
        if max_arg not in arg:
            sim_map[i]=max_arg
            arg.add(i)
            arg.add(max_arg)

In [ ]:
final_sim_map={}
for key in sim_map:
    if key not in final_sim_map:
        other_keys = []
        k=key
        while k in sim_map:
            other_keys.append(sim_map[k])
            k=sim_map[k]

        for ok in other_keys:
            final_sim_map[ok] = key


In [ ]:
k=10

In [ ]:
topk=[]
for i in tqdm(range(len(cosine_scores))):
    s1=BW_sents[i]
    top_k=np.argsort(cosine_scores[i].cpu().numpy())[::-1][:k]
    choices = [(CR_sents[j],cosine_scores[i][j].cpu().item()) for j in top_k]
    topk.append([s1]+choices)

100%|██████████| 8830/8830 [00:30<00:00, 287.24it/s]


In [ ]:
topk_df=pd.DataFrame(topk,columns=['BW']+[f'CR{i}' for i in range(1,11)])
topk_df

,BW,CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9,CR10
0,CDC does not currently recommend the use of fa...,(People shouldn't wear face-masks to prevent t...,(Facemasks are ineffective to block transmissi...,(Face masks should only be worn by medical pro...,(A study conducted by the Centers for Disease ...,(WHO does not recommend wearing masks during C...,(The Centers for Disease Control and Preventio...,(OSHA training shows some face masks can do mo...,(Face Masks Alone Can Help In Preventing COVID...,(Coronavirus (COVID-19) is large in size where...,(A Centers for Disease Control report released...
1,@RonaldKlain Election Fraud was inevitable. De...,(widespread election fraud occurred during the...,(The US Election was rigged against Donald Tru...,(Voter Fraud Ring Caught Rigging Elections for...,(The US Election has been rigged to favour Joe...,(Democrats are colluding with the CIA to alter...,(CNN and ABC showed proof of Georgia election ...,"(Election Night Coup D'Ã©tat Plot Exposed!, 0....",(Surveillance video captured widespread voter ...,"(Vote rigging in poll fraud in US, 0.577522218...",(Trump's electoral fraud tweets and the public...
2,Donald Trump is the first president in modern ...,(President Donald Trump claim that Spanish flu...,(Donald Trump was the first U.S. president to ...,(The month of November is going to be the USAâ...,(Donald Trump &quot;saved&quot; the Veterans D...,"(Donald Trump declared martial law., 0.5028006...",(Eric Trump tweeted about the airstrike that k...,(U.S. President Donald Trump inherited a &quot...,(Joe Biden signed blank executive orders on hi...,(there were no mass shootings while former Pre...,(Trump remarked in the first US presidential d...
3,"First they came for our Free Speech, then they...","(“Antifa” posted an ad on the Lincoln, Nebrask...",(Starbucks and KFC offering free food and drin...,"(Amazon pays ""little or no taxes to state & lo...",(Federal taxpayers are “subsidizing” Amazon’s ...,"(""Currently, when we ship products out of Amer...",(Democratic Party Now Selling Official Riot Su...,"(""I have stated my concerns with Amazon long b...",(McDonald's removes their American flags in su...,"(""We’re charging China tariffs. We’ve never ta...","(""Amazon is doing great damage to tax paying r..."
4,"Yes, I want somebody to shoot Nancy Pelosi in ...","(""White House to Consider Impeaching Nancy Pel...","(Kamala Harris:âAfter We Impeach, We Round U...",(IG Report: Hillary Clinton has committed 'Sex...,(“When I did it [established a program to pros...,"(Trump Planning Mass Arrests, Military Tribuna...",(Trump To Release The Names Of Sexual Predator...,"(Does the NRA Make Gun Targets of Children, 0....","(""WIKILEAKS: OBAMA RAN PEDOPHILE RING OUT OF W...",(Elizabeth Warren Wants Criminal Penalties For...,(Democratic New York Rep. Alexandria Ocasio-Co...
...,...,...,...,...,...,...,...,...,...,...,...
8825,Surprise surprise - Fauci lied again\n\nAnd I ...,(CORONAVIRUS : SCIENTISTS SAY IT'S A BIO WEAPO...,(A Facebook post from April 4 claim that a US ...,"(while talking about funding a Wuhan, China, l...",(Dr. Anthony Fauci resigned from the White Hou...,(Blogpost claims coronavirus outbreak is a cov...,(PRESIDENT TRUMP ANNOUNCED THE BIOLOGICAL LAB...,"(Lab-Made Coronavirus Triggers Debate, 0.64559...",(Trump accounced Roche Diagnostics has success...,"(Chief Science Officer for Pfizer Says ""Second...",(Dr. Anthony Fauci has resigned from the White...
8826,Trump was at ground zero two days after the at...,(Two days after the 9/11 attack Trump was at G...,(Says Donald Trump went to Ground Zero two day...,"(Rep. Jim Jordan said ""While Obama and Biden w...",(Donald Trump wore a 9/11 costume to a 9/11 me...,"(Says Donald Trump ""cashed in"" on Sept. 11, ""...","(""When Moammar Gadhafi was set to visit the Un...",(Donald Trump &quot;saved&quot; the Veterans D...,(U.S. President Donald Trump performed several...,(Video shows US soldiers greeting Donald Trump...,(Says he didn't cancel a politic

In [ ]:
topk_df['Score']=topk_df.CR1.apply(lambda x: np.round(x[1],1))

In [ ]:
topk_df.shape

(8830, 12)

In [ ]:
topk_df_filt=topk_df[topk_df.CR1.apply(lambda x: True if np.round(x[1],1)>=0.5 else False)]

In [ ]:
topk_df_filt.shape

(7817, 12)

In [ ]:
topk_df_filt

,BW,CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9,CR10,Score
0,CDC does not currently recommend the use of fa...,(People shouldn't wear face-masks to prevent t...,(Facemasks are ineffective to block transmissi...,(Face masks should only be worn by medical pro...,(A study conducted by the Centers for Disease ...,(WHO does not recommend wearing masks during C...,(The Centers for Disease Control and Preventio...,(OSHA training shows some face masks can do mo...,(Face Masks Alone Can Help In Preventing COVID...,(Coronavirus (COVID-19) is large in size where...,(A Centers for Disease Control report released...,0.8
1,@RonaldKlain Election Fraud was inevitable. De...,(widespread election fraud occurred during the...,(The US Election was rigged against Donald Tru...,(Voter Fraud Ring Caught Rigging Elections for...,(The US Election has been rigged to favour Joe...,(Democrats are colluding with the CIA to alter...,(CNN and ABC showed proof of Georgia election ...,"(Election Night Coup D'Ã©tat Plot Exposed!, 0....",(Surveillance video captured widespread voter ...,"(Vote rigging in poll fraud in US, 0.577522158...",(Trump's electoral fraud tweets and the public...,0.6
2,Donald Trump is the first president in modern ...,(President Donald Trump claim that Spanish flu...,(Donald Trump was the first U.S. president to ...,(The month of November is going to be the USAâ...,(Donald Trump &quot;saved&quot; the Veterans D...,"(Donald Trump declared martial law., 0.5028006...",(Eric Trump tweeted about the airstrike that k...,(U.S. President Donald Trump inherited a &quot...,(Joe Biden signed blank executive orders on hi...,(there were no mass shootings while former Pre...,(Trump remarked in the first US presidential d...,0.5
3,"First they came for our Free Speech, then they...","(“Antifa” posted an ad on the Lincoln, Nebrask...",(Starbucks and KFC offering free food and drin...,"(Amazon pays ""little or no taxes to state & lo...",(Federal taxpayers are “subsidizing” Amazon’s ...,"(""Currently, when we ship products out of Amer...",(Democratic Party Now Selling Official Riot Su...,"(""I have stated my concerns with Amazon long b...",(McDonald's removes their American flags in su...,"(""We’re charging China tariffs. We’ve never ta...","(""Amazon is doing great damage to tax paying r...",0.5
5,insider at #Robinhood just sent me these inter...,(the image shows the online trading platform R...,"(""Trump-approved 'M.A.G.A. checks'"" are availa...","(Tweet by Warren Buffett on Bitcoin, 0.4651251...","(""In her campaign for president, Hillary Clint...","(""New reports show Kevin Nicholson made over $...","(The CEO of Wendy&#039;s donated about $400,00...",(Donald Trump and Mike Pence want to gamble wi...,(The Trump campaign sent out a fundraiser emai...,(Tweets posted by @satishmaneshnde Rhea Chakra...,(Brick Pallets For Riots From ACME BRICK CO Ow...,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
8825,Surprise surprise - Fauci lied again\n\nAnd I ...,(CORONAVIRUS : SCIENTISTS SAY IT'S A BIO WEAPO...,(A Facebook post from April 4 claim that a US ...,"(while talking about funding a Wuhan, China, l...",(Dr. Anthony Fauci resigned from the White Hou...,(Blogpost claims coronavirus outbreak is a cov...,(PRESIDENT TRUMP ANNOUNCED THE BIOLOGICAL LAB...,"(Lab-Made Coronavirus Triggers Debate, 0.64559...",(Trump accounced Roche Diagnostics has success...,"(Chief Science Officer for Pfizer Says ""Second...",(Dr. Anthony Fauci has resigned from the White...,0.7
8826,Trump was at ground zero two days after the at...,(Two days after the 9/11 attack Trump was at G...,(Says Donald Trump went to Ground Zero two day...,"(Rep. Jim Jordan said ""While Obama and Biden w...",(Donald Trump wore a 9/11 costume to a 9/11 me...,"(Says Donald Trump ""cashed in"" on Sept. 11, ""...","(""When Moammar Gadhafi was set to visit the Un...",(Donald Trump &quot;saved&quot; the Veterans D...,(U.S. President Donald Trump performed several...,(Video shows US soldiers greeting Donald Trump..